In [1]:
import numpy as np #matrix math
import math
import tensorflow as tf #machine learning
import matplotlib.pyplot as plt #plotting

%matplotlib inline

In [2]:
attempt=8
num_channels = 4
len_edge = 64
bs = 16
n_filters = [num_channels, 128, 128, 128, 256, 256]
h_dim = 128
latent_dim = 10 * 10

inedge = len_edge / (2 ** (len(n_filters)-1))
latvisdim = int(np.sqrt(latent_dim))
tb_imgs_to_display = 4

train_size = 3200
valid_size = 16

recording_interval = 10
epochs = 1000000

testlogpath = './vae_logs2/test/' + str(attempt)
trainlogpath = './vae_logs2/train/' + str(attempt)

In [3]:
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

#filename = '/run/media/ron/silver_small/twelve_months/3d/S1A_IW_GRDH_1SDV_20160325T083601_20160325T083630_010523_00FA23_6F51.tif'
#filename = '/home/ron/Documents/local51/S1A_IW_GRDH_1SDV_20160325T083601_20160325T083630_010523_00FA23_6F51.tif'
#filename='/media/ramdisk/lil_s2.tif'
filename = '/home/ron/Downloads/lil_s2.tif'
#filename = '/run/media/ron/silver_small/twelve_months/3d/S1A_IW_GRDH_1SDV_20160325T083601_20160325T083630_010523_00FA23_6F51.tif'
#filename='/media/ramdisk/smaller.tif'

In [4]:
def sample_a_batch(filebase, batchsize, tilesize=128, sb=0, normalize=False, flattened=True):
    # can sample for spatio-temporal (single_file = False), and spatial-only case (single_file = True).
    import numpy as np
    from osgeo import gdal
    
    if '.' in filebase:
        single_file = True
        S = gdal.Open(filebase)
    else:
        single_file = False
        S = gdal.Open(filebase + '_1.vrt')
        
    samples = []
    
    if single_file:
        while len(samples) < batchsize:
            RX = np.random.randint(S.RasterXSize-tilesize,size=1)
            RY = np.random.randint(S.RasterYSize-tilesize,size=1)    
            
            if sb:
                B = S.GetRasterBand(sb)
                A = np.transpose(B.ReadAsArray(RX[0], RY[0], tilesize, tilesize), dtype=np.float32)                
                #print np.min(A)
                if np.min(A) > 0:
                    if normalize:
                        A = A / normalize[2][sb-1]                        
                    A = np.expand_dims(A, 2)                    
                    if flattened:
                        A = A.flatten()                        
                    samples.append(A)
            else:
                A = np.transpose(S.ReadAsArray(RX[0], RY[0], tilesize, tilesize)).astype(np.float32)
                if np.min(A) > 0:
                    if normalize:
                        for b in range(A.shape[2]):
                            A[:, :, b] = A[:, :, b] / float(normalize[2][b])
                        if flattened:
                            A = A.flatten()
                            
                        samples.append(A)
        
    else: # must be overhauled
        while len(samples) < batchsize:
            RX = np.random.randint(S.RasterXSize-tilesize,size=1)
            RY = np.random.randint(S.RasterYSize-tilesize,size=1) 
            
            skip_loc = False
            months = []
            
            for m in range(1,13):
                S = gdal.Open(filebase + '_' + str(m) + '.vrt')
                A = np.transpose(S.ReadAsArray(RX[i], RY[i], tilesize, tilesize))
                if np.min(A) == 0.0:
                    skip_loc = True
                    break
                else:
                    months.append(A)                
            if not skip_loc:
                months = np.array(months)
                samples.append(months)
        
    return np.array(samples)

def normalization_parameters(fn):
    from osgeo import gdal
    S = gdal.Open(fn)
    mns = []
    sds = []
    maxs = []
    
    for b in range(S.RasterCount):
        B = S.GetRasterBand(b+1)
        mn, sd = B.ComputeStatistics(1)[2:4]
        mns.append(mn)
        sds.append(sd)
        maxs.append(B.GetMaximum())
        
    return mns, sds, maxs

In [5]:
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)
    
def fc_layer(inp, channels_in, channels_out, name='fc'):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([channels_in, channels_out]), name='W')
        b = tf.Variable(tf.zeros([channels_out]), name='B')
        return tf.nn.relu(tf.matmul(inp, w) + b)

In [6]:
#switch from relu to elu
with tf.name_scope('Input'):
    X = tf.placeholder(tf.float32, shape=([None, len_edge, len_edge, num_channels]))#bs
    if num_channels == 2:
        X_show = tf.concat([X, tf.expand_dims(X[:, :, :, 1], 3)], axis=3)
        tf.summary.image('input_images', X_show, max_outputs=tb_imgs_to_display)
    elif num_channels > 3:
        tf.summary.image('input_images', X[:, :, :, 0:3], max_outputs=tb_imgs_to_display)
    else:
        tf.summary.image('input_images', X, max_outputs=tb_imgs_to_display)

cur_input = X

Ws = []    
shapes = []

for l, n_out in enumerate(n_filters[1:]):
    n_input = cur_input.get_shape().as_list()[3]
    shapes.append(cur_input.get_shape().as_list())
    with tf.name_scope('conv_indecon_' + str(l)):
        with tf.variable_scope(str(l)):
            W = tf.get_variable('weights', [3, 3, n_input, n_out], initializer=tf.contrib.layers.xavier_initializer_conv2d())#, name='weights')
            b = tf.Variable(tf.zeros([n_out]), name='bias')
            tf.summary.histogram('weights', W)
            tf.summary.histogram('biases', b)
            Ws.append(W)
            conv = tf.nn.conv2d(cur_input, W, strides=[1, 2, 2, 1], padding='SAME')
            act = tf.nn.relu(conv+b)#tf.nn.relu(conv,b)
            tf.summary.histogram('activations', act)
        cur_input = act

with tf.name_scope('Dense'):
    with tf.name_scope('Fully_Encode'):
        flattened = tf.reshape(cur_input, [-1, inedge * inedge * n_filters[-1]])
        W_enc = tf.get_variable('W_enc', [inedge * inedge * n_filters[-1], h_dim], initializer=tf.contrib.layers.xavier_initializer())
        b_enc = bias_variable([h_dim], 'b_enc')        
        full1 = tf.nn.relu(tf.matmul(flattened, W_enc) + b_enc)
        #dropout here
        tf.summary.histogram('weights', W_enc)
        tf.summary.histogram('biases', b_enc)
        tf.summary.histogram('activations', full1)
    with tf.name_scope('Mu'):
        W_mu = tf.get_variable('W_mu', [h_dim, latent_dim], initializer=tf.contrib.layers.xavier_initializer())
        b_mu = bias_variable([latent_dim], 'b_mu')
        mu = tf.matmul(full1, W_mu) + b_mu
    with tf.name_scope('Logstd'):
        W_logstd = tf.get_variable('W_logstd', [h_dim, latent_dim], initializer=tf.contrib.layers.xavier_initializer())
        b_logstd = bias_variable([latent_dim], 'b_logstd')
        logstd = tf.matmul(full1, W_logstd) + b_logstd
    with tf.name_scope('VAE_final'):
        noise = tf.random_normal([1, latent_dim])
        z = mu + tf.multiply(noise, tf.exp(.5*logstd))
    with tf.name_scope('Fully_Decode'):
        
        W_dec1 = tf.get_variable('W_dec1', [latent_dim, h_dim], initializer=tf.contrib.layers.xavier_initializer())
        b_dec1 = bias_variable([h_dim], 'b_dec1')
        
        full2 = tf.nn.relu(tf.matmul(z, W_dec1) + b_dec1)
        #dropout here, 0.5
        
        tf.summary.histogram('weights_1', W_dec1)
        tf.summary.histogram('biases_1', b_dec1)
        tf.summary.histogram('activations_1', full2)
        
        W_dec2 = tf.get_variable('W_dec2', [h_dim, inedge * inedge * n_filters[-1]], initializer=tf.contrib.layers.xavier_initializer())
        b_dec2 = bias_variable([inedge * inedge * n_filters[-1]], 'b_dec2')
        
        full3 = tf.nn.relu(tf.matmul(full2, W_dec2) + b_dec2)
        
        tf.summary.histogram('weights_2', W_dec2)
        tf.summary.histogram('biases_2', b_dec2)
        tf.summary.histogram('activations_2', full3)
        reshaped = tf.reshape(full3, [-1, inedge, inedge, n_filters[-1]])
        
z_visual = tf.reshape(z, [-1, latvisdim, latvisdim, 1])
tf.summary.image('latents', z_visual, max_outputs=tb_imgs_to_display)
tf.summary.histogram('Latent', z)

Ws.reverse()
shapes.reverse()
cur_input = reshaped

for l, shape in enumerate(shapes):
    cur_name = 'deconv' + str(l)  
    W = Ws[l]    
    with tf.name_scope('conv_indecon_' + str(len(Ws)-(l+1))):
        b = tf.Variable(tf.zeros([W.get_shape().as_list()[2]]), name='bias_dec_'+str(l))
        dec = tf.nn.conv2d_transpose(cur_input, W, tf.stack([bs, shape[1], shape[2], shape[3]]), strides=[1,2,2,1], padding='SAME')
        if l+1 < len(shapes):
            act = tf.nn.relu(dec+b)
            cur_input = act

with tf.name_scope('reconst'):
    reconstruction = tf.nn.sigmoid(dec + b)
    if num_channels == 2:
        r_show = tf.concat([reconstruction, tf.expand_dims(reconstruction[:, :, :, 1], 3)], axis=3)
        tf.summary.image('reconstructed_images', r_show, max_outputs=tb_imgs_to_display)
    elif num_channels > 3:
        tf.summary.image('reconstructed_images', reconstruction[:, :, :, 0:3], max_outputs=tb_imgs_to_display)
    else:
        tf.summary.image('reconstructed_images', reconstruction, max_outputs=tb_imgs_to_display)

In [7]:
X_flat = tf.contrib.layers.flatten(X)
R_flat = tf.contrib.layers.flatten(reconstruction)

log_likelihood = tf.reduce_sum(X_flat*tf.log(R_flat + 1e-9)+(1 - X_flat)*tf.log(1 - R_flat + 1e-9), reduction_indices=1)
tf.summary.scalar('LogLike', tf.reduce_mean(log_likelihood))

KL_term = -.5*tf.reduce_sum(1 + 2*logstd - tf.pow(mu,2) - tf.exp(2*logstd), reduction_indices=1)
tf.summary.scalar('KL', tf.reduce_mean(KL_term))

variational_lower_bound = tf.reduce_mean(log_likelihood - KL_term)
tf.summary.scalar('cost', variational_lower_bound)

validator = tf.cast(variational_lower_bound, tf.int32) # alibi
optimizer = tf.train.AdamOptimizer(0.01).minimize(-variational_lower_bound)

In [9]:
merged_summary = tf.summary.merge_all()
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)
saver = tf.train.Saver()
train_writer = tf.summary.FileWriter(trainlogpath)
valid_writer = tf.summary.FileWriter(testlogpath)
train_writer.add_graph(sess.graph)

In [10]:
stats = normalization_parameters(filename)
X_all = sample_a_batch(filename, train_size, len_edge, sb=None, normalize=stats, flattened=False)
x_valid = sample_a_batch(filename, valid_size, len_edge, sb=None, normalize=stats, flattened=False)

In [12]:
step = bs
ep = 0
for ep in range(epochs):
    x_batch = X_all[step-bs:step]
    
    if (ep%recording_interval == 0):
        vvv, s = sess.run([validator, merged_summary], feed_dict={X: x_valid})
        valid_writer.add_summary(s, ep)
        
    _, s = sess.run([optimizer, merged_summary], feed_dict={X: x_batch})
    
    if (ep%recording_interval == 0):
        train_writer.add_summary(s, ep)
        
    step += bs
    
    if step == train_size:
        step = bs

KeyboardInterrupt: 

save_path = saver.save(sess, "./test_save.ckpt")

sess = tf.Session()
new_saver = tf.train.import_meta_graph('test_save.ckpt.meta')

new_saver.restore(sess, tf.train.latest_checkpoint('./'))

all_vars = tf.get_collection('vars')
for v in all_vars:
    v_ = sess.run(v)
    print(v_)

#decoder test:
Z_to_decode = 
_full2 = tf.nn.relu(tf.matmul(Z_to_decode, W_dec1) + b_dec1)
_full3 = tf.nn.relu(tf.matmul(_full2, W_dec2) + b_dec2)
_reshaped = tf.reshape(_full3, [-1, inedge, inedge, n_filters[-1]])

_cur_input = _reshaped

for l, shape in enumerate(shapes):
    W = Ws[l]
    _dec = tf.nn.conv2d_transpose(_cur_input, W, tf.stack([bs, shape[1], shape[2], shape[3]]), strides=[1,2,2,1], padding='SAME')
    if l+1 < len(shapes):
        act = tf.nn.relu(_dec+b)
        cur_input = act
        
_reconstruction = tf.nn.sigmoid(_dec + b)
    if num_channels == 2:
        r_show = tf.concat([_reconstruction, tf.expand_dims(_reconstruction[:, :, :, 1], 3)], axis=3)
        tf.summary.image('reconstructed_images', r_show, max_outputs=tb_imgs_to_display)
    elif num_channels > 3:
        tf.summary.image('reconstructed_images', _reconstruction[:, :, :, 0:3], max_outputs=tb_imgs_to_display)
    else:
        tf.summary.image('reconstructed_images', _reconstruction, max_outputs=tb_imgs_to_display)

**IGNORE FOLLOWING CELLS:**

import pickle
cifars = []
for i in range(1, 6):
    with open('/home/ron/Downloads/cifar-10-batches-py/data_batch_' + str(i), mode='rb') as f:
        data = pickle.load(f)
        raw_images = data[b'data']
        cls = np.array(data[b'labels'])        
        raw_float = np.array(raw_images, dtype=float) / 255.0
        images = raw_float.reshape([-1, 3, len_edge, len_edge])
        images = images.transpose([0, 2, 3, 1])        
        cifars = cifars + list(images)        
cifars = np.array(cifars)